<a href="https://colab.research.google.com/github/GB-studyingcode/Cost-Optimization/blob/main/Cost_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gurobipy
import gurobipy as gp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 107.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
# Set up parameter

# Tập hợp Plant
I = 3

# Tập hợp WH
K = 2

# Tập hợp Customer
J = 4
## Tham số 1: chi phí --> Set up cost matrices

cost_ik = [[4,7],[8,5],[5,6]]  # ma trận chi phí giao hàng từ plant đến Warehouse
cost_kj = [[6,4,8,4],[3,6,7,7]]  # ma trận chi phí giao hàng từ Warehouse đến Customer

## Tham số 2: năng lực cung ứng --> Set up supply capacity matrix
s_i = [440,650,410]

## Tham số 3: nhu cầu khách hàng --> Set up demand matrix
d_j = [300,300,250,400]


# Set up Model
m = gp.Model('Transshipment_Proble')



Restricted license - for non-production use only - expires 2026-11-23


In [ ]:
# Khai báo biến
# Biến 1: Lượng giao từ Plant đến WH
x = m.addVars(len(cost_ik),len(cost_ik[0]),lb = 0,vtype = gp.GRB.INTEGER, name = 'x')

# Biến 2: Lượng giao hàng từ WH đến Customer
y = m.addVars(len(cost_kj),len(cost_kj[0]),lb=0,vtype = gp.GRB.INTEGER, name = 'y')

In [ ]:
# Hàm mục tiêu (Objective Function)
m.setObjective(gp.quicksum(x[i,k] * cost_ik[i][k] for i in range(I) for k in range(K)) +
               gp.quicksum(y[k,j] * cost_kj[k][j] for k in range(K) for j in range(J)),
               gp.GRB.MINIMIZE
)

In [ ]:
# Ràng buộc (Constraints)
# Supply Contraints
# Với mỗi nhà máy , tổng lượng giao ra không được vượt quá năng lực của nhà máy đó
for i in range(I):
  m.addConstr(gp.quicksum(x[i,k] for k  in range(K)) <= s_i[i], name = 'Supply Constraint')

# Demand Constraint
# Với mỗi khách hàng, tổng lượng nhận vào phải cao hơn nhu cầu
for j in range(J):
    m.addConstr(gp.quicksum(y[k,j] for k in range(K))>= d_j[j],name ='Demand Constraint')

# Transshipment Constraint
# Với mỗi kho , tổng lượng hàng vào phải lớn hơn tổng lượng hàng ra
for k in range(K):
    m.addConstr(gp.quicksum(x[i,k] for i in range(I)) >= gp.quicksum(y[k,j] for j in range(J)), name = 'Transshipment Constraint')
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 14 columns and 28 nonzeros
Model fingerprint: 0x2d52576a
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+02]
Found heuristic solution: objective 13610.000000
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 28 nonzeros
Variable types: 0 continuous, 14 integer (0 binary)

Root relaxation: objective 1.126000e+04, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    11260.000000 11260.0000  0.00%

In [ ]:
for v in m.getVars():
  print(f"{v.VarName} ={v.x} ")

x[0,0] =440.0 
x[0,1] =-0.0 
x[1,0] =-0.0 
x[1,1] =550.0 
x[2,0] =260.0 
x[2,1] =-0.0 
y[0,0] =-0.0 
y[0,1] =300.0 
y[0,2] =-0.0 
y[0,3] =400.0 
y[1,0] =300.0 
y[1,1] =-0.0 
y[1,2] =250.0 
y[1,3] =-0.0 


In [ ]:
plant_to_WH =[]
for i in range(I):
  for k in range(K):
    print(f"x[{i},{k}] = {x[i,k].x}")
    plant_to_WH.append(x[i,k].x)

df =  pd.DataFrame(plant_to_WH)
df

x[0,0] = 440.0
x[0,1] = -0.0
x[1,0] = -0.0
x[1,1] = 550.0
x[2,0] = 260.0
x[2,1] = -0.0


,0
0,440.0
1,-0.0
2,-0.0
3,550.0
4,260.0
5,-0.0


In [ ]:
for k in range(K):
  for j in range(J):
    print(f"y[{k},{j}] = {y[k,j].x}")

y[0,0] = -0.0
y[0,1] = 300.0
y[0,2] = -0.0
y[0,3] = 400.0
y[1,0] = 300.0
y[1,1] = -0.0
y[1,2] = 250.0
y[1,3] = -0.0
